#Phần 1: Giới thiệu và Cài đặt

In [1]:
# Cài đặt thư viện spaCy
!pip install -U spacy

# Tải mô hình tiếng Anh (en_core_web_md)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 32.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#Phần 2: Phân tích câu và Trực quan hóa

##2.1. Tải mô hình và phân tích câu

In [2]:
import spacy
from spacy import displacy

# Tải mô hình đã cài đặt
# Sử dụng en_core_web_md vì nó chứa vector từ và cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")

In [12]:
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."

# Phân tích câu qua pipeline
doc = nlp(text)

##2.2. Trực quan hóa cây phụ thuộc

In [13]:
# Trực quan hóa cây phụ thuộc
options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
displacy.render(doc, style="dep", jupyter=True, options=options)

In [10]:
displacy.serve(doc, style="dep")

/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


Dựa trên biểu đồ trực quan hóa ở trên:


1. Từ nào là gốc (ROOT) của câu?

- Trả lời: Từ "jumps".

2. "jumps" có những từ phụ thuộc (dependent) nào? Các quan hệ đó là gì?

- Trả lời:

  - "fox": quan hệ nsubj (chủ ngữ danh từ).

  - "over": quan hệ prep (giới từ).

  - ".": quan hệ punct (dấu câu).


3. fox là head của những từ nào?

- Trả lời: fox là head của The (det), quick (amod), và brown (amod).

#Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [11]:
text_2 = "Apple is looking at buying U.K. startup for $1 billion"
doc_2 = nlp(text_2)

# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc_2:
    children = [child.text for child in token.children]

    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


#Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

##4.1. Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [5]:
text_3 = "The cat chased the mouse and the dog watched them."
doc_3 = nlp(text_3)

print("--- Kết quả trích xuất bộ ba (Subject, Verb, Object) ---")

for token in doc_3:
    # Chỉ tìm các động từ (VERB)
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""

        # Duyệt qua các con của động từ để tìm nsubj và dobj
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

--- Kết quả trích xuất bộ ba (Subject, Verb, Object) ---
Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


##4.2. Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [6]:
text_4 = "The big, fluffy white cat is sleeping on the warm mat."
doc_4 = nlp(text_4)

print("\n--- Kết quả trích xuất Danh từ và Tính từ bổ nghĩa ---")

for token in doc_4:
    # Chỉ tìm các danh từ (NOUN)
    if token.pos_ == "NOUN":
        adjectives = []

        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)

        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")


--- Kết quả trích xuất Danh từ và Tính từ bổ nghĩa ---
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


#Phần 5: Bài tập tự luyện

##Bài 1: Tìm động từ chính của câu

In [7]:
def find_main_verb(doc):
    for token in doc:
        # Động từ chính thường có nhãn dependency là ROOT
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
        # Trường hợp ROOT không phải VERB (ví dụ câu danh từ), ta vẫn trả về ROOT
        elif token.dep_ == "ROOT":
            return token
    return None

# Test
test_doc = nlp("The cat chased the mouse.")
main_verb = find_main_verb(test_doc)
print(f"Động từ chính: {main_verb.text} (POS: {main_verb.pos_})")

Động từ chính: chased (POS: VERB)


##Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

In [8]:
def extract_noun_chunks_manual(doc):
    chunks = []
    for token in doc:
        # Bắt đầu từ một danh từ
        if token.pos_ == "NOUN":
            # Lấy các từ con bên trái (thường là det, amod, compound) và chính nó
            chunk_tokens = []

            # Duyệt cây con (subtree) của danh từ để lấy trọn vẹn cụm từ
            # Hoặc cách đơn giản hơn là duyệt children:
            current_chunk = [child.text for child in token.lefts if child.dep_ in ["det", "amod", "compound"]]
            current_chunk.append(token.text)

            # Ghép lại thành chuỗi
            if current_chunk:
                chunks.append(" ".join(current_chunk))
    return chunks

# Test
text_noun = "The quick brown fox jumps over the lazy dog."
doc_noun = nlp(text_noun)

print("Manual Extraction:", extract_noun_chunks_manual(doc_noun))
print("Spacy Built-in:", [chunk.text for chunk in doc_noun.noun_chunks]) # Để so sánh

Manual Extraction: ['The quick brown fox', 'the lazy dog']
Spacy Built-in: ['The quick brown fox', 'the lazy dog']


##Bài 3: Tìm đường đi ngắn nhất trong cây

In [9]:
def get_path_to_root(token):
    path = [token]
    current_token = token

    # Duyệt ngược lên head cho đến khi gặp token mà head là chính nó (Gốc của cây)
    # Hoặc until current_token.dep_ == "ROOT"
    while current_token.head != current_token:
        current_token = current_token.head
        path.append(current_token)

    return path

# Test
# Lấy từ "brown" trong câu ví dụ để tìm đường về gốc "jumps"
# Câu: The quick brown fox jumps... (brown -> fox -> jumps)
target_token = doc_noun[2] # brown
path = get_path_to_root(target_token)

print(f"Đường đi từ '{target_token.text}' đến ROOT:")
print(" -> ".join([t.text for t in path]))

Đường đi từ 'brown' đến ROOT:
brown -> fox -> jumps
